In [12]:
import pandas as pd

In [13]:
unigram = pd.read_csv('unigram_binary_False.csv')
unigram = unigram[unigram.caption == "unigrams ('middle part', ExtractMiddlePart(processing=None))"]

In [14]:
se = pd.read_csv('binary_False.csv')
se = se[se.caption == "infersent ('middle part', ExtractMiddlePart(processing=None))"]

In [15]:
se_unique = se[~se.sentence.isin(unigram.sentence.values.tolist())].copy()
se_unique['origin'] = 'sentence embedding'

In [16]:
uni_unique = unigram[~unigram.sentence.isin(se.sentence.values.tolist())].copy()
uni_unique['origin'] = 'unigrams'

In [26]:
se_multi = se[se.sentence.isin(unigram.sentence.values.tolist())].copy()
uni_multi =  unigram[unigram.sentence.isin(se.sentence.values.tolist())].copy()
uni_multi['origin'] = 'uni'
se_multi['origin'] = 'se'

In [18]:
print('Unigram Multi {} | SE Multi {} '.format(len(uni_multi), len(se_multi)))
print('Unigram Unique {} | SE Unique {} '.format(len(uni_unique), len(se_unique)))
print('Uni {} | SE {}'.format(len(unigram),len(se)))

Unigram Multi 661 | SE Multi 661 
Unigram Unique 174 | SE Unique 164 
Uni 835 | SE 825


In [19]:
errors = pd.concat([uni_multi,se_unique,uni_unique])[['sentence', 'object_a', 'object_b', 'predicted','gold', 'origin']]

In [23]:
errors.to_csv('unigram_se_errors.csv')

In [22]:
errors.origin.value_counts()

both                  661
unigrams              174
sentence embedding    164
Name: origin, dtype: int64

In [32]:
different = []
for i, row in se_multi.iterrows():
    predicted = row['predicted']
    uni_sentence = uni_multi[uni_multi.sentence == row['sentence']]
    u_p = row['predicted']
    print('{}\t{}\t{}'.format(row['sentence'],predicted,u_p))
    if predicted != u_p:
        different.append(row['sentence'])
print(len(different))

In summary - a plaigarized technology - a bad cement and a worse concrete.	NONE	NONE
I have also rented many cars, American and Japanese, and with the exception of a very smooth ride in a Cadillac, I always like the Japanese cars better (Nissan and Toyota, vs. Ford, Chrysler and GM).	NONE	NONE
Apple would be better off competing with Nintendo than Sony and MS.	BETTER	BETTER
Come on now Windows 8 may not be Windows 7 in popularity but it is actually a better, faster and more secure OS.	WORSE	WORSE
Python is much better suited as a "glue" language, while Java is better characterized as a low-level implementation language.	BETTER	BETTER
It's a decent rival to the Mercedes-Benz CLA45 AMG, though the Mercedes still wins when it comes to power output; let's hope Audi hurries up and brings out a new RS 3.	NONE	NONE
How was it that Nokia overcame Motorola?	NONE	NONE
Instead of pizza, it'll be easier to choose a salad or healthy sandwich .	NONE	NONE
We believe that your upgrade experience in Wi

That's a problem for Dell, and it won't get any better until it starts beating Apple at its own game.	NONE	NONE
Perl would have been more better had there been good REPL like Python.	BETTER	BETTER
Last time I checked Apple had 3% market share (Personal Computers) Sony does better.	NONE	NONE
Latex paint provides better fade resistance, flexibility, chalk resistance and adhesion.	NONE	NONE
OS/2 was technically superior, but Microsoft beat IBM through marketing and by tying its Office applications to Windows."	NONE	NONE
-SQL Server is significantly superior to MySQL, faster than PostgreSQL, and for most businesses is cheaper than Oracle.	BETTER	BETTER
Docter continued her solid play in Saturday's home defeat of cornell, leading harvard with 13 kills.	NONE	NONE
It's better for Amazon to cannibalize itself than have it done anymore by Apple," he adds.	BETTER	BETTER
Try shooting a wedding on a manual focus Hasselblad and a Nikon F3 or worse still a Leica ;)	NONE	NONE
Intel PCs can do art and

Thanks to Apple removing Google maps which is 1000x better.	NONE	NONE
Are you saying that you use JavaScript outside a browser, and that you feel it is a better language in that context than Java, C++, C, Smalltalk, Lisp, Perl, Python, etc?	BETTER	BETTER
Even if this juice isn't your typical cider, it's just as good if not better in our opinion!	NONE	NONE
@tanasten Bowling is WAY better, baseball is WAY better, golf is WAY better, tennis is the same and boxing is the drizzlin' poopiedoodoocacapoopledoopledoggiepoopledoops!!!!!	BETTER	BETTER
We chose for PostgreSQL (not to start an MySQL pg flamewar, but pg was the more suitable for our purposes) because it's open, and thus safer.	NONE	NONE
I was using google music but amazon is just better.	NONE	NONE
Windows 8 Developer desktop - looks like Windows 7, runs a hair faster.	NONE	NONE
That's true, function passing is painless in Python and a little harder in Ruby.	NONE	NONE
Its 362 nit brightness level is one of the best we've seen, improv

amazon had a similar service for its books but google is much faster and simpler.	NONE	NONE
Coca-Cola per say is not worse than any other type of cola, like Pepsi, or generic branded cola.	WORSE	WORSE
The better young male athletes in the US, said McEnroe, prefer to play football and basketball over tennis.	NONE	NONE
juice" is ok, but "cider" is much better.	NONE	NONE
While Motorola has diminished, Samsung has excelled	NONE	NONE
Most of the applications I see that use Oracle would work fine or better with an OSS database such as PostgreSQL or even SQLite.	NONE	NONE
Also, keep in tmind that two years ago, Dell,HP,Gateway etc, never used AMD chips as Intel produced the better chip.	NONE	NONE
Would the new Nikon 85 be that much better than the old one or the inexpensive Sigma to justify the investment?	BETTER	BETTER
Mazda 6, Honda Accord Euro and Toyota Camry better watch out for this advanced strategic move from Ford.	NONE	NONE
football was lousy, basketball was worse and baseball -- tho